In [1]:
import pandas as pd
import re
import requests

In [2]:
df = pd.read_csv("./data/병합_청약매물_목록_정보.csv", encoding='cp949')

C:\Users\ilhoo\AppData\Local\Temp\ipykernel_3320\254017625.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/병합_청약매물_목록_정보.csv", encoding='cp949')


### 고민

1. 블록들어간 단어, 공공택지지구  -> 전처리 필수  
2. (마전동), (경기도 의왕시 고천동 190-1일원) -> 처럼 괄호안에 동이 들어간 경우도 있음    
3. 경기도 수원시 장안구 이목동 -> 얘처럼 경기도는 3depth가 안통할 수도    


In [3]:
# 중복된 '공고번호' 제거 (첫 번째 항목만 남김)
df_copy = df.reset_index(drop=True).drop_duplicates(subset="공고번호", keep='first', ignore_index=True)
    
# '정제_공급위치' 열 생성 후 간단한 전처리
df_copy["정제_공급위치"] = df_copy["공급위치"].apply(lambda address: re.sub(r'\b\w*(블록|BL)\w*\b', '', address).replace('공공택지지구', '').strip())    

1. 주소 모양 통일 (도로명 주소들 -> 지번 주소로)

In [5]:
pd.set_option('display.max_rows', None)

# 도로명 주소 키워드 정의
road_keywords = ['길', '로', '대로']

# 도로명 주소만 포함하는 행 필터링
addrs = df_copy[df_copy['정제_공급위치'].apply(lambda x: any(keyword in x for keyword in road_keywords))]

addrs.정제_공급위치

0                                     경기도 부천시 원미구 신흥로 20
1                        경기도 성남시 분당구 산운로 126 (운중동, 산운마을)
3                          경기도 평택시 청북읍 청북남로 277 (부영사랑으로)
48                         경기도 성남시 분당구 산운로 126 (산운마을9단지)
64               경기도 성남시 분당구 판교원로 209 (판교동, 판교원마을6단지아파트)
87                  인천검단시 서구 불로동 484-3번지 일대(검단신도시 AB21-)
127                          서울특별시 구로구 개봉동 369-1, 312-38
129                   경기도 평택시 현덕면 화양리 평택화양지구 도시개발사업  1로트
161                             인천광역시 서구 불로동 검단신도시 AB20-
169                 서울특별시 양천구 신목로2길 7-12 (신정동, 어반클라쎄 목동)
172              서울특별시 강동구 길동 386-4 (길동, 에스아이팰리스 강동 센텀2)
181                                   인천광역시 서구 불로동 검단신도시
182                             서울특별시 강서구 강서로 353 (내발산동)
184                              인천광역시 서구 왕길동 133-3번지 일대
200              경기도 화성시 동탄중앙로 119-14 (반송동, 동탄 센트럴 포레스트)
201                  경기도 오산시 수목원로 577-15 (세교동, 세마역 에듀파크)
202                      경기도 오산시 여계산로 60 (금암동, 세교 센트럴파크)
203                            

In [7]:

# API 키 입력
api_key = "66262ed5fa70195418d041e708940e58"

locations = []

for addr in addrs['정제_공급위치']:
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        place = response.json()['documents']
        locations.append(place)
    else:
        print(f"주소 '{addr}'에 대한 요청 실패: {response.status_code}")
        locations.append([])  # 실패한 경우 빈 리스트 추가

# 지번 주소 추출
city = []
town = []
for i in range(len(locations)):
    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_name'))
    except (IndexError, AttributeError):
        city.append('없음')
        town.append('없음')

# 결과 확인
for i in range(len(city)):
    print(f"주소: {addrs['정제_공급위치'].iloc[i]}, 시: {city[i]}, 동: {town[i]}")


주소: 경기도 부천시 원미구 신흥로 20, 시: 부천시 원미구, 동: 심곡동
주소: 경기도 성남시 분당구 산운로 126 (운중동, 산운마을), 시: 성남시 분당구, 동: 운중동
주소: 경기도 평택시 청북읍 청북남로 277 (부영사랑으로), 시: 평택시, 동: 청북읍 옥길리
주소: 경기도 성남시 분당구 산운로 126 (산운마을9단지), 시: 성남시 분당구, 동: 운중동
주소: 경기도 성남시 분당구 판교원로 209 (판교동, 판교원마을6단지아파트), 시: 성남시 분당구, 동: 판교동
주소: 인천검단시 서구 불로동 484-3번지 일대(검단신도시 AB21-), 시: 없음, 동: 없음
주소: 서울특별시 구로구 개봉동 369-1, 312-38, 시: 없음, 동: 없음
주소: 경기도 평택시 현덕면 화양리 평택화양지구 도시개발사업  1로트, 시: 없음, 동: 없음
주소: 인천광역시 서구 불로동 검단신도시 AB20-, 시: 없음, 동: 없음
주소: 서울특별시 양천구 신목로2길 7-12 (신정동, 어반클라쎄 목동), 시: 양천구, 동: 신정동
주소: 서울특별시 강동구 길동 386-4 (길동, 에스아이팰리스 강동 센텀2), 시: 강동구, 동: 길동
주소: 인천광역시 서구 불로동 검단신도시, 시: 없음, 동: 없음
주소: 서울특별시 강서구 강서로 353 (내발산동), 시: 강서구, 동: 내발산동
주소: 인천광역시 서구 왕길동 133-3번지 일대, 시: 서구, 동: 왕길동
주소: 경기도 화성시 동탄중앙로 119-14 (반송동, 동탄 센트럴 포레스트), 시: 화성시, 동: 반송동
주소: 경기도 오산시 수목원로 577-15 (세교동, 세마역 에듀파크), 시: 오산시, 동: 세교동
주소: 경기도 오산시 여계산로 60 (금암동, 세교 센트럴파크), 시: 오산시, 동: 금암동
주소: 경기도 부천시 신흥로 20 (심곡동), 시: 부천시 원미구, 동: 심곡동
주소: 인천광역시 서구 불로동 검단신도시  공동주택용지, 시: 없음, 동: 없음
주소: 인천광역시 연수구 해돋이로 160-19 (송도동),

도로명 주소를 가진 주소는 괄호 안에 동이 존재하는 경우가 많음  
-> 그럼 위도 경도 추출하고  
-> 시군구 읍면동까지 업데이트 해서 위도 경도 뽑아냄    
-> 공급 위치 기준으로 도로명 주소들 행만 꺼내서 괄호안에 동으로 끝나는 단어를 읍면동 행에 넣어줌 (다시 업데이트 과정)  
-> 괄호 안에 없는 것은 어떻게 해결? : 읍면동 none인 행들만 위에 api 코드 돌려봄  
-> (마무리1) 시군구에 동이 박혀있으면 읍면동으로 옮겨주는 과정  
-> (마무리2, 이건 조금 고민) 시도에 군,구 박혀있으면 읍면동으로 옮겨주는 과정 (가평군이 시도에 박혀있는 경구가 있었음)

  
  
### 질문
시군구 보다 행정동이 중요한가?

### to do
해당 과정 오전 시간에 수행

In [ ]:
kakao_api_key = '66262ed5fa70195418d041e708940e58'

def pre_col_gen(df):

    # 중복된 '공고번호' 제거 (첫 번째 항목만 남김)
    df_copy = df.reset_index(drop=True).drop_duplicates(subset="공고번호", keep='first', ignore_index=True)
    
    # '정제_공급위치' 열 생성 후 간단한 전처리
    df_copy["정제_공급위치"] = df_copy["공급위치"].apply(lambda address: re.sub(r'\b\w*(블록|BL)\w*\b', '', address).replace('공공택지지구', '').strip())
    
    # 필요한 열만 선택
    df_copy = df_copy[["공고번호", "정제_공급위치"]]
    
    return df_copy

def process_address_data(df):
    # 카카오맵 API 사용해서 위도 경도 추출
    kakao_api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    def get_lat_lon_kakao(address):
        try:
            response = requests.get(kakao_api_url, headers=headers, params={"query": address})
            if response.status_code == 200:
                result = response.json()
                if result['documents']:
                    lat = result['documents'][0]['y']
                    lon = result['documents'][0]['x']
                    return lat, lon
            return None, None
        except Exception as e:
            print(f"Error fetching data for address {address}: {e}")
            return None, None

    df['위도'], df['경도'] = zip(*df['정제_공급위치'].apply(lambda x: get_lat_lon_kakao(x)))

    return df

def process_address(df):
    
    def split_and_update_address(address):
        parts = address.split(' ')
        
        # 주소가 "경기도"로 시작할 경우, "경기도"를 제거
        if parts[0] == "경기도":
            parts = parts[1:]
        
        if len(parts) >= 3:
            sido = parts[0]
            sigungu = parts[1]
            eupmyeondong = parts[2]
            updated_address = f"{sido} {sigungu} {eupmyeondong}"
            return sido, sigungu, eupmyeondong, updated_address
        else:
            return None, None, None, address
    
    # 주소 분리 및 업데이트
    df[['시도', '시군구', '읍면동', '정제_공급위치_업데이트']] = df['정제_공급위치'].apply(lambda x: pd.Series(split_and_update_address(x)))
    
    # 기존 정제_공급위치 열을 업데이트함
    df['정제_공급위치'] = df['정제_공급위치_업데이트']
    
    # 불필요한 열 제거
    df = df.drop(columns=['정제_공급위치_업데이트'])
    
    return df

def lat_lon_pipeline(df):

    # 서울, 경기, 인천 지역만 필터링
    df = df[df['공급지역명'].isin(['서울', '경기', '인천'])]

    df_copy = pre_col_gen(df)

    # Kakao API로 위도와 경도 가져오기
    df_copy = process_address_data(df_copy)
    
    # 주소 분리 및 업데이트 
    df_copy = process_address(df_copy)

    # 위도와 경도가 None인 행만 선택하여 다시 처리
    df_none = df_copy[(df_copy['위도'].isnull()) & (df_copy['경도'].isnull())].copy()
    
    if not df_none.empty:
        df_none_processed = process_address_data(df_none)
        df_copy.loc[df_none.index, ['위도', '경도']] = df_none_processed[['위도', '경도']]
    
    df_copy.drop("정제_공급위치", axis=1, inplace=True)
    
    return df, df_copy


In [ ]:
df = pd.read_csv("./data/병합_청약매물_목록_정보.csv", encoding='cp949')
df, df_copy = lat_lon_pipeline(df)
df_merged = pd.merge(df, df_copy, on='공고번호', how='left')